In [3]:
import pandas as pd

variable,total_u050_u19_private,with_u050_u19_private,no_u050_u19_private,total_u050_1964_private,with_u050_1964_private,no_u050_1964_private,total_u050_o65_private,with_u050_o65_private,no_u050_o65_private,total_050099_u19_private,...,no_300399_o65_public,total_o400_u19_public,with_o400_u19_public,no_o400_u19_public,total_o400_1964_public,with_o400_1964_public,no_o400_1964_public,total_o400_o65_public,with_o400_o65_public,no_o400_o65_public
Alabama,118574,16011,102563,204512,77663,126849,40102,17399,22703,127143,...,5077,286990,32492,254498,1057473,80523,976950,313726,298715,15011
Alaska,11100,1356,9744,21800,4921,16879,2786,1005,1781,10652,...,571,71836,7735,64101,210158,23414,186744,50613,48032,2581
Arizona,129901,21626,108275,265159,83644,181515,69742,29034,40708,125307,...,7337,497492,45858,451634,1791805,142601,1649204,599732,566967,32765
Arkansas,76135,7381,68754,124212,33646,90566,21202,8320,12882,73304,...,1766,155877,17177,138700,571358,47685,523673,168136,160990,7146
California,587773,119066,468707,1369085,428667,940418,290210,93988,196222,719061,...,42582,3185031,366834,2818197,11135862,1030157,10105705,2912561,2665116,247445
Colorado,69549,15762,53787,174111,72857,101254,34635,13418,21217,64725,...,5117,546505,50139,496366,1912828,138764,1774064,445433,415675,29758
Connecticut,53996,11042,42954,106978,31184,75794,26909,11088,15821,46731,...,3968,336544,29149,307395,1172026,83765,1088261,341459,315124,26335
Delaware,13893,2808,11085,27701,10385,17316,7364,3592,3772,18875,...,846,81029,8041,72988,272816,21968,250848,102712,97929,4783
District of Columbia,14295,1363,12932,32149,7976,24173,6519,2761,3758,7387,...,624,69112,7717,61395,276899,17100,259799,44362,40458,3904
Florida,325740,62757,262983,749480,290117,459363,240220,97631,142589,390532,...,31805,1315698,138289,1177409,5239025,364697,4874328,1908742,1811311,97431


In [2]:
# Income data cleaning
income = pd.read_csv('income.csv', index_col=0, thousands=',').T

# ChatGPT: regex handling, gave me the structur for the regex and the filter lines

# Create total columns
income_suffixes = ['u050', '050099', '100137', '138149', '150199', '200249', '250299', '300399', 'o400']
age_suffixes = ['u19', '1964', 'o65']
type_suffixes = ['private', 'public']

# Sum across all levels of all variables
totals_to_sum = income.filter(regex=f'total').columns
income[f'total_all_all_all'] = income[totals_to_sum].sum(axis=1)

withs_to_sum = income.filter(regex=f'with').columns
income[f'with_all_all_all'] = income[withs_to_sum].sum(axis=1)

# For each insurance type
for ins_type in type_suffixes:
    # Sum across all levels with the insurance type
    totals_to_sum = income.filter(regex=f'total.*{ins_type}').columns
    income[f'total_all_all_{ins_type}'] = income[totals_to_sum].sum(axis=1)

    withs_to_sum = income.filter(regex=f'with.*{ins_type}').columns
    income[f'with_all_all_{ins_type}'] = income[withs_to_sum].sum(axis=1)

    # Sum across all ages for the insurance type and each income level
    for inc in income_suffixes:
        totals_to_sum = income.filter(regex=f'total.*{inc}.*{ins_type}').columns
        income[f'total_{inc}_all_{ins_type}'] = income[totals_to_sum].sum(axis=1)

        withs_to_sum = income.filter(regex=f'with.*{inc}.*{ins_type}').columns
        income[f'with_{inc}_all_{ins_type}'] = income[withs_to_sum].sum(axis=1)

# For each age
for age in age_suffixes:
    # Sum across all levels with the age group
    totals_to_sum = income.filter(regex=f'total.*{age}').columns
    income[f'total_all_{age}_all'] = income[totals_to_sum].sum(axis=1)

    withs_to_sum = income.filter(regex=f'with.*{age}').columns
    income[f'with_all_{age}_all'] = income[withs_to_sum].sum(axis=1)
    
    # Sum across all income levels for the age and each insurance type
    for ins_type in type_suffixes:
        totals_to_sum = income.filter(regex=f'total.*{age}.*{ins_type}').columns
        income[f'total_all_{age}_{ins_type}'] = income[totals_to_sum].sum(axis=1)

        withs_to_sum = income.filter(regex=f'with.*{age}.*{ins_type}').columns
        income[f'with_all_{age}_{ins_type}'] = income[withs_to_sum].sum(axis=1)

# For each income level
for inc in income_suffixes:
    # Sum across all levels with the income level
    totals_to_sum = income.filter(regex=f'total.*{inc}').columns
    income[f'total_{inc}_all_all'] = income[totals_to_sum].sum(axis=1)

    withs_to_sum = income.filter(regex=f'with.*{inc}').columns
    income[f'with_{inc}_all_all'] = income[withs_to_sum].sum(axis=1)

    # Sum across all insurance types for the age and each income level 
    for age in age_suffixes:
        totals_to_sum = income.filter(regex=f'total.*{inc}.*{age}').columns
        income[f'total_{inc}_{age}_all'] = income[totals_to_sum].sum(axis=1)

        withs_to_sum = income.filter(regex=f'with.*{inc}.*{age}').columns
        income[f'with_{inc}_{age}_all'] = income[withs_to_sum].sum(axis=1)


# Create pct variable as with / total for every combination of levels
pct_cols = []

for suffix in income.columns.str.extract(r'^.*?_(.*)', expand=False).unique():
    total_col = income.filter(regex=f'total_{suffix}').columns
    with_col = income.filter(regex=f'with_{suffix}').columns
    
    pct_col = f'pct_{suffix}'

    income[pct_col] = income[with_col[0]] / income[total_col[0]]


# Split out into individual variables for each of the dropdowns
data_to_use = income.filter(regex=f'pct_').copy()

data_to_use['state'] = income.index

melted = data_to_use.melt(id_vars= "state",
                        var_name="variable", 
                        value_name="value")

melted[['pct', 'level', 'age', 'type']] = melted['variable'].str.split('_', expand=True)

melted = melted.drop(columns=['variable', 'pct'])

melted.to_csv('income_clean.csv') 

/var/folders/3x/jkmnd88d4l723jvw0jkwhbph0000gn/T/ipykernel_95876/645501412.py:67: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  income[f'with_{inc}_{age}_all'] = income[withs_to_sum].sum(axis=1)
/var/folders/3x/jkmnd88d4l723jvw0jkwhbph0000gn/T/ipykernel_95876/645501412.py:56: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  income[f'total_{inc}_all_all'] = income[totals_to_sum].sum(axis=1)
/var/folders/3x/jkmnd88d4l723jvw0jkwhbph0000gn/T/ipykernel_95876/645501412.py:59: PerformanceWarning: DataFrame is highly fragmented.  This is u

,state,value,level,age,type
0,Alabama,0.135030,u050,u19,private
1,Alaska,0.122162,u050,u19,private
2,Arizona,0.166481,u050,u19,private
3,Arkansas,0.096946,u050,u19,private
4,California,0.202571,u050,u19,private
...,...,...,...,...,...
6235,Washington,0.793285,o400,o65,all
6236,West Virginia,0.841284,o400,o65,all
6237,Wisconsin,0.784502,o400,o65,all
6238,Wyoming,0.817707,o400,o65,all


In [10]:
# Race/ethnicity data cleaning
race_eth = pd.read_csv('race_ethnicity.csv', index_col=0, thousands=',').T

# ChatGPT: regex handling, gave me the structur for the regex and the filter lines

# Create total columns
race_suffixes = ['white', 'black', 'asian', 'multi', 'hispanic']
age_suffixes = ['u6', '618', '1925', '2634', '3544', '4554', '5564', '6574', 'o75']

# Sum across all levels of all variables
totals_to_sum = race_eth.filter(regex=f'total').columns
race_eth[f'total_all_all'] = race_eth[totals_to_sum].sum(axis=1)

withs_to_sum = race_eth.filter(regex=f'with').columns
race_eth[f'with_all_all'] = race_eth[withs_to_sum].sum(axis=1)

# For each race/ethinicity group
for race in race_suffixes:
        # Sum across all levels with the race/ethnicity group
        totals_to_sum = race_eth.filter(regex=f'total.*{race}').columns
        race_eth[f'total_all_{race}'] = race_eth[totals_to_sum].sum(axis=1)

        withs_to_sum = race_eth.filter(regex=f'with.*{race}').columns
        race_eth[f'with_all_{race}'] = race_eth[withs_to_sum].sum(axis=1)

# For each age group
for age in age_suffixes:
        # Sum across all levels with the age group
        totals_to_sum = race_eth.filter(regex=f'total.*{age}').columns
        race_eth[f'total_{age}_all'] = race_eth[totals_to_sum].sum(axis=1)

        withs_to_sum = race_eth.filter(regex=f'with.*{age}').columns
        race_eth[f'with_{age}_all'] = race_eth[withs_to_sum].sum(axis=1)


# Create pct variable as with / total for every combination of levels
pct_cols = []

for suffix in race_eth.columns.str.extract(r'^.*?_(.*)', expand=False).unique():
    total_col = race_eth.filter(regex=f'total_{suffix}').columns
    with_col = race_eth.filter(regex=f'with_{suffix}').columns
    
    pct_col = f'pct_{suffix}'

    race_eth[pct_col] = race_eth[with_col[0]] / race_eth[total_col[0]]


# Split out into individual variables for each of the dropdowns
data_to_use = race_eth.filter(regex=f'pct_').copy()

data_to_use['state'] = race_eth.index

melted = data_to_use.melt(id_vars= "state",
                        var_name="variable", 
                        value_name="value")

melted[['pct', 'age', 'level']] = melted['variable'].str.split('_', expand=True)

melted = melted.drop(columns=['variable', 'pct'])

melted.to_csv('race_ethnicity_clean.csv') 

In [11]:
# Insurance type data cleaning
insurance_type = pd.read_csv('insurance_type.csv', index_col=0, thousands=',').T

# ChatGPT: regex handling, gave me the structur for the regex and the filter lines

# Create total columns
gender_suffixes = ['male', 'female']
age_suffixes = ['u6', '618', '1925', '2634', '3544', '4554', '5564', '6574', 'o75']
type_suffixes = ['private', 'public', 'employer', 'medicare', 'medicaid', ]

# Sum across all levels of all variables
totals_to_sum = insurance_type.filter(regex=f'total').columns
insurance_type[f'total_all_all_all'] = insurance_type[totals_to_sum].sum(axis=1)

withs_to_sum = insurance_type.filter(regex=f'with').columns
insurance_type[f'with_all_all_all'] = insurance_type[withs_to_sum].sum(axis=1)

# For each insurance type
for ins_type in type_suffixes:
    # Sum across all levels with the insurance type
    totals_to_sum = insurance_type.filter(regex=f'total.*{ins_type}').columns
    insurance_type[f'total_all_all_{ins_type}'] = insurance_type[totals_to_sum].sum(axis=1)

    withs_to_sum = insurance_type.filter(regex=f'with.*{ins_type}').columns
    insurance_type[f'with_all_all_{ins_type}'] = insurance_type[withs_to_sum].sum(axis=1)

    # Sum across all ages for the insurance type and each gender
    for gender in gender_suffixes:
        totals_to_sum = insurance_type.filter(regex=f'total.*{gender}.*{ins_type}').columns
        insurance_type[f'total_all_{gender}_{ins_type}'] = insurance_type[totals_to_sum].sum(axis=1)

        withs_to_sum = insurance_type.filter(regex=f'with.*{gender}.*{ins_type}').columns
        insurance_type[f'with_all_{gender}_{ins_type}'] = insurance_type[withs_to_sum].sum(axis=1)

# For each age group
for age in age_suffixes:
    # Sum across all levels with the age
    totals_to_sum = insurance_type.filter(regex=f'total.*{age}').columns
    insurance_type[f'total_{age}_all_all'] = insurance_type[totals_to_sum].sum(axis=1)

    withs_to_sum = insurance_type.filter(regex=f'with.*{age}').columns
    insurance_type[f'with_{age}_all_all'] = insurance_type[withs_to_sum].sum(axis=1)

    # Sum across all genders for the age and each insurance type
    for ins_type in type_suffixes:
        totals_to_sum = insurance_type.filter(regex=f'total.*{age}.*{ins_type}').columns
        insurance_type[f'total_{age}_all_{ins_type}'] = insurance_type[totals_to_sum].sum(axis=1)

        withs_to_sum = insurance_type.filter(regex=f'with.*{age}.*{ins_type}').columns
        insurance_type[f'with_{age}_all_{ins_type}'] = insurance_type[withs_to_sum].sum(axis=1)

# For each gender
for gender in gender_suffixes:
    # Sum across all levels with the gender
    totals_to_sum = insurance_type.filter(regex=f'total.*{gender}').columns
    insurance_type[f'total_all_{gender}_all'] = insurance_type[totals_to_sum].sum(axis=1)

    withs_to_sum = insurance_type.filter(regex=f'with.*{gender}').columns
    insurance_type[f'with_all_{gender}_all'] = insurance_type[withs_to_sum].sum(axis=1)

    # Sum across all insurance types for the gender and each age group
    for age in age_suffixes:
        totals_to_sum = insurance_type.filter(regex=f'total.*{age}.*{gender}').columns
        insurance_type[f'total_{age}_{gender}_all'] = insurance_type[totals_to_sum].sum(axis=1)

        withs_to_sum = insurance_type.filter(regex=f'with.*{age}.*{gender}').columns
        insurance_type[f'with_{age}_{gender}_all'] = insurance_type[withs_to_sum].sum(axis=1)


# Create pct variable as with / total for every combination of levels
pct_cols = []

for suffix in insurance_type.columns.str.extract(r'^.*?_(.*)', expand=False).unique():
    total_col = insurance_type.filter(regex=f'total_{suffix}').columns
    with_col = insurance_type.filter(regex=f'with_{suffix}').columns
    
    pct_col = f'pct_{suffix}'

    insurance_type[pct_col] = insurance_type[with_col[0]] / insurance_type[total_col[0]]


# Split out into individual variables for each of the dropdowns
data_to_use = insurance_type.filter(regex=f'pct_').copy()

data_to_use['state'] = insurance_type.index

melted = data_to_use.melt(id_vars= "state",
                        var_name="variable", 
                        value_name="value")

melted[['pct', 'age', 'gender', 'level']] = melted['variable'].str.split('_', expand=True)

melted = melted.drop(columns=['variable', 'pct'])

melted.to_csv('insurance_type_clean.csv') 

/var/folders/3x/jkmnd88d4l723jvw0jkwhbph0000gn/T/ipykernel_95876/418157176.py:50: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  insurance_type[f'with_{age}_all_{ins_type}'] = insurance_type[withs_to_sum].sum(axis=1)
/var/folders/3x/jkmnd88d4l723jvw0jkwhbph0000gn/T/ipykernel_95876/418157176.py:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  insurance_type[f'total_{age}_all_{ins_type}'] = insurance_type[totals_to_sum].sum(axis=1)
/var/folders/3x/jkmnd88d4l723jvw0jkwhbph0000gn/T/ipykernel_95876/418157176.py:50: PerformanceWarning

,state,value,age,gender,level
0,Alabama,0.533482,u6,male,private
1,Alaska,0.605891,u6,male,private
2,Arizona,0.577334,u6,male,private
3,Arkansas,0.469879,u6,male,private
4,California,0.585479,u6,male,private
...,...,...,...,...,...
9355,Washington,0.592616,o75,female,all
9356,West Virginia,0.620932,o75,female,all
9357,Wisconsin,0.579256,o75,female,all
9358,Wyoming,0.576225,o75,female,all
